<a href="https://colab.research.google.com/github/ahmetyuce526/yolo_shhep_detection/blob/main/yolo_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# -------------------------------Video ----------------------------------

In [ ]:
# import numpy and opencv
import numpy as np
import cv2
​
​# Define labels list
labels = ["Sheep",]

# Define box colors 
colors = [255,0,0]

# Load the model and weights from working directory using opencv dnn module
model = cv2.dnn.readNet("yolov3_testing.cfg", "yolov3_sheep.weights")
​
# Get all layer names from model
layers = model.getLayerNames()
​
# Get output layer from model
output_layers = []
for layer in model.getUnconnectedOutLayers():
    index = layer[0]-1
    output_layers.append(layers[index])
​
# Load the test video from working directory as .mp4 format
capture = cv2.VideoCapture("sheep1.mp4")

# Get frame width and height values
width_cap= int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height_cap= int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
​
# Create a video codec for save the detectioned video
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('custom_sheep_detection2.mp4', fourcc, 20.0, (width_cap,  height_cap))

while True:
    # Read frames from video file
    ret, frame = capture.read()
    
    # Check is there any frame
    if ret == 0:
        break
    
    # Get width an height value for boxes
    width = frame.shape[1]
    height = frame.shape[0]

    # Convert images to blob format image = [416,416,3] ==> blob = [1,3,416,416]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416,416),swapRB = True, crop = False)

    # Send the blob image into model
    model.setInput(blob)

    # Get with detectioned layer
    detection_layers = model.forward(output_layers)
​
    ##################### Non Maximum Supression ######################
​
    # Create lists for ids, confidece and boxes
    ids_list = list()
    boxes_list = list()
    confidence_list = list()
​    
    # Check all detectioned layers
    for detection_layer in detection_layers:
        for object_detection in detection_layer:

            # Get class values from list
            scores = object_detection[5:]
​            
            # Get max values index number from scores
            predicted_id = np.argmax(scores)

​            # Get max values from scores
            confidence = scores[predicted_id]
​
​            # Check confidence values
            if confidence > 0.3:

                # Get class name from labels list
                label = labels[predicted_id]

                # Get boxes from detection list
                bounding_box = object_detection[0:4]*np.array([width,height, width, height])

​                # Specify the x,y, box_width, and box_height of detectioned box
                (box_center_x, box_center_y, box_width, box_height) = bounding_box.astype("int")
​
                start_x =  int(box_center_x - box_width/2)
                start_y =  int(box_center_y - box_height/2)
​                
                # Send the values to the was created lists
                ids_list.append(predicted_id)
                confidence_list.append(float(confidence))
                boxes_list.append([start_x, start_y, int(box_width), int(box_height)])
​   
    # Get max values of the detectioned objects to avoid more than one times detection the an object

    ##################### Non Maximum Supression ######################
    
    max_ids = cv2.dnn.NMSBoxes(boxes_list, confidence_list, 0.5,0.4)
    for max_id in max_ids:
        # Get max id and boxes from max_ids list
        max_class_id = max_id[0]
        box = boxes_list[max_class_id]
​        
        # Specify start_x, start_y, box_width, box_height of the boxes
        start_x = box[0]
        start_y = box[1]
        box_width = box[2]
        box_height = box[3]
​        
        # Find values of predicted_id, label, confidence
        predicted_id = ids_list[max_class_id]
        label = labels[ids_list[max_class_id]]
        confidence = confidence_list[max_class_id]

​        # Specify end_x, end_y of the boxes
        end_x = start_x + box_width
        end_y = start_y + box_height

​        # Get color from colors list
        color = colors[predicted_id]
​        # Define the class name
        label = " {}: {:.2f}%".format(label,confidence)
​        
        # Draw a rectange box on the image
        cv2.rectangle(frame, (start_x,start_y),(end_x,end_y),color, 3)

        # Put a text on the image
        cv2.putText(frame, label, (start_x, start_y-10), cv2.FONT_HERSHEY_SIMPLEX,2, [0,0,255], 2)  
        cv2.putText(frame, "Ahmet Yuce", (30, 40), cv2.FONT_HERSHEY_SIMPLEX,1, [255,255,255], 2)
​    
    # Save the detectioned video to the working directory
    out.write(frame)
    # Show the video on the screen
    cv2.imshow("frame", frame)
    if cv2.waitKey(29) & 0xFF == ord("q"):
        break
​# Close all open windows
capture.release()
out.release()
cv2.destroyAllWindows()

# -------------------------------Video ----------------------------------